In [1]:
!pip install stellargraph

In [2]:
import os 
import pandas as pd
from stellargraph import StellarGraph 

Mounted at /content/drive


In [ ]:
DATA_PATH=os.path.join(os.getcwd(),'DATA/')
SAVE_PATH=os.path.join(os.getcwd(),'saved_model/')
Result_PATH=os.path.join(os.getcwd(),'Result/')

In [3]:
edges_data=pd.read_csv(DATA_PATH+'sample.csv')
node_feature=pd.read_csv(DATA_PATH+'sample_feature.csv')
#node_feature=node_feature.drop_duplicates(subset=['index'])
node_features = node_feature.set_index(keys=['index'], inplace=False, drop=True)

In [4]:
node_features.head()

,1,2,3,4,5,6,7,8,9,10,...,290,291,292,293,294,295,296,297,298,299
index,,,,,,,,,,,,,,,,,,,,,
5887,-0.168308,0.453490,-0.095535,0.290504,-0.894610,0.103735,-0.280093,0.074014,0.503019,0.367744,...,-0.163552,-0.086980,-0.145552,0.070703,0.141626,0.001629,-0.096676,-0.052995,-0.112178,-0.090999
3426,2.397051,0.090396,1.236613,0.839081,0.093752,0.323479,-0.483196,-0.176435,-0.552185,0.748059,...,0.584843,0.119463,-0.409786,0.380476,-0.362136,0.192507,0.413073,-0.362811,-0.028254,0.358826
6340,0.844389,2.615271,0.093406,-1.072956,-0.955502,0.727380,-0.141938,-0.092633,0.751066,-1.077170,...,0.114912,-0.324140,-0.153945,0.141717,0.199205,-0.725853,-0.691025,0.233834,0.390434,-0.268641
7031,-0.560994,0.060390,-0.311671,0.110356,0.077707,0.342163,-0.432374,0.023661,0.243273,-0.051051,...,0.007316,0.084201,0.145957,0.006628,-0.108482,-0.102031,0.008283,-0.010563,0.212869,0.000766
7994,2.597059,3.824801,1.423339,1.348240,-2.891339,-0.556217,0.250086,-0.015109,1.021333,0.098333,...,-0.350584,-1.176642,1.377007,-1.279261,1.606988,-0.253019,0.147770,1.038439,0.548416,-1.134808


In [5]:
edges_data=edges_data.drop_duplicates()
edges_data.head()

,target,source
0,2,3
1,3,4
2,4,5
3,5,6
4,6,7


In [6]:
sample = StellarGraph({"feature": node_features}, {"edge": edges_data})

In [7]:
print(sample.info())


StellarGraph: Undirected multigraph
 Nodes: 8472, Edges: 267926

 Node types:
  feature: [8472]
    Features: float32 vector, length 299
    Edge types: feature-edge->feature

 Edge types:
    feature-edge->feature: [267926]
        Weights: all 1 (default)
        Features: none


In [8]:
import stellargraph as sg
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, HinSAGE, link_classification

from tensorflow import keras
from sklearn import preprocessing, feature_extraction, model_selection

from stellargraph import globalvar
from stellargraph import datasets
from IPython.display import display, HTML
%matplotlib inline

In [9]:
G=sample
sg.random.set_seed(1161)

In [10]:
# Define an edge splitter on the original graph G:
edge_splitter_test = EdgeSplitter(G)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G, and obtain the
# reduced graph G_test with the sampled links removed:
G_test, edge_ids_test, edge_labels_test = edge_splitter_test.train_test_split(
    p=0.1, method="global", keep_connected=True
)

** Sampled 26792 positive and 26792 negative edges. **


In [11]:
# Define an edge splitter on the reduced graph G_test:
edge_splitter_train = EdgeSplitter(G_test)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G_test, and obtain the
# reduced graph G_train with the sampled links removed:
G_train, edge_ids_train, edge_labels_train = edge_splitter_train.train_test_split(
    p=0.1, method="global", keep_connected=True
)

** Sampled 24113 positive and 24113 negative edges. **


In [12]:

print(G_train.info())

print(G_test.info())

StellarGraph: Undirected multigraph
 Nodes: 8472, Edges: 217021

 Node types:
  feature: [8472]
    Features: float32 vector, length 299
    Edge types: feature-edge->feature

 Edge types:
    feature-edge->feature: [217021]
        Weights: all 1 (default)
        Features: none
StellarGraph: Undirected multigraph
 Nodes: 8472, Edges: 241134

 Node types:
  feature: [8472]
    Features: float32 vector, length 299
    Edge types: feature-edge->feature

 Edge types:
    feature-edge->feature: [241134]
        Weights: all 1 (default)
        Features: none


In [13]:
batch_size = 20
epochs = 200

In [14]:
num_samples = [20, 10]

In [15]:
train_gen = GraphSAGELinkGenerator(G_train, batch_size, num_samples)
train_flow = train_gen.flow(edge_ids_train, edge_labels_train, shuffle=True)

In [16]:
test_gen = GraphSAGELinkGenerator(G_test, batch_size, num_samples)
test_flow = test_gen.flow(edge_ids_test, edge_labels_test)

In [17]:
layer_sizes = [64, 64]
graphsage = GraphSAGE(
    layer_sizes=layer_sizes, generator=train_gen, bias=True, dropout=0.3
)

In [18]:
# Build the model and expose input and output sockets of graphsage model
# for link prediction
x_inp, x_out = graphsage.in_out_tensors()

In [19]:
prediction = link_classification(
    output_dim=1, output_act="sigmoid", edge_embedding_method="ip"
)(x_out)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [20]:
model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(lr=0.01),
    loss=keras.losses.binary_crossentropy,
    metrics=["acc"],
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [21]:
init_train_metrics = model.evaluate(train_flow)
init_test_metrics = model.evaluate(test_flow)

print("\nTrain Set Metrics of the initial (untrained) model:")
for name, val in zip(model.metrics_names, init_train_metrics):
    print("\t{}: {:0.4f}".format(name, val))

print("\nTest Set Metrics of the initial (untrained) model:")
for name, val in zip(model.metrics_names, init_test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

2680/2680 [==============================] - 108s 40ms/step - loss: 0.7012 - acc: 0.5302

Train Set Metrics of the initial (untrained) model:
	loss: 0.7002
	acc: 0.5315

Test Set Metrics of the initial (untrained) model:
	loss: 0.7012
	acc: 0.5302


In [22]:
history = model.fit(train_flow, epochs=epochs, validation_data=test_flow, verbose=2,use_multiprocessing=ㅆ겨)
model.save(SAVE_PATH+'sage.model')

Epoch 1/200
2412/2412 - 194s - loss: 0.6031 - acc: 0.6832 - val_loss: 0.5839 - val_acc: 0.7021 - 194s/epoch - 80ms/step
Epoch 2/200


KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt
fig=sg.utils.plot_history(history,return_figure=True)
fig.savefig(Result_PATH+'sage_64_64')

In [ ]:
train_metrics = model.evaluate(train_flow)
test_metrics = model.evaluate(test_flow)

print("\nTrain Set Metrics of the trained model:")
for name, val in zip(model.metrics_names, train_metrics):
    print("\t{}: {:0.4f}".format(name, val))

print("\nTest Set Metrics of the trained model:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))